In [1]:
print("Hello")

Hello


In [5]:
import numpy
import cohere
import redis 

from redis import Redis
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
from redis.commands.search.result import Result

import lxml

In [ ]:
import cohere
from redis import Redis 

vector_field_name = 'encoded_text_block'

# Connect to Cohere API
cohere_client = cohere.Client("g4DkrTsekiIckvuuqKcKRguDVkHQLbHN03VlD2r9")

# Set up Redis connect
redis_conn = Redis( host = 'redis-13894.c300.eu-central-1-1.ec2.cloud.redislabs.com', 
                    port=13894, 
                    password='LCpWGSMAwsi00jOj2iCVVouXBvOcn008')

# Function to encode blocks using Cohere
def encode_blocks(blocks):
    return cohere_client.embed(texts=blocks,  model='small', truncate='LEFT').embeddings


# Function to save encoded blocks into Redis
def save_into_redis(client:Redis, colection_id: str, documents: list, vector_field_name: str):    
    p = client.pipeline(transaction=False)
    for document in documents:   
        blocks = split_into_blocks(document['render'])
        encoded_blocks = encode_blocks(blocks)
        for i in range(len(encoded_blocks)):
            #hash key
            key = f"{colection_id}:{document['path']}:{str(i)}"
        
            #hash values
            document_metadata = {}
            document_metadata['encoded_text_block'] = np.array(encoded_blocks[i]).astype(np.float32).tobytes()
            # HSET
            p.hset(key, mapping=document_metadata)
            #p.set(key, np.array(encoded_blocks[i]).astype(np.float32).tobytes())
            
    p.execute()

save_into_redis( redis_conn, "Collection_1", list_of_dicts, vector_field_name)

import struct

#redis_conn
# get all keys
keys = redis_conn.keys('*')
print(keys[0])

# get a value of a key
value = redis_conn.get('Collection_1:daily-journaling-app:34')
print(value)
# calculate the number of 4-byte floats in the data
num_floats = len(value) // 4

# construct the format string
format_str = 'f' * num_floats

values = struct.unpack(format_str, value)

print(values)

def create_flat_index (redis_conn, vector_field_name, number_of_vectors, vector_dimensions=1024, distance_metric='L2'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "FLAT", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric, "INITIAL_CAP": number_of_vectors, "BLOCK_SIZE":number_of_vectors }),
        TagField("product_type"),
        TextField("item_name"),
        TextField("item_keywords"),
        TagField("country")        
    ])

ITEM_KEYWORD_EMBEDDING_FIELD='item_keyword_vector'
TEXT_EMBEDDING_DIMENSION=768
NUMBER_PRODUCTS=1000

#create flat index & load vectors
create_flat_index(redis_conn, ITEM_KEYWORD_EMBEDDING_FIELD, NUMBER_PRODUCTS, TEXT_EMBEDDING_DIMENSION,'COSINE')